In [10]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

In [3]:
data_dir = './sounds'
audio_files = glob(data_dir + '/*')

In [4]:
len(audio_files)

0

In [13]:
data_dir = './sounds'
audio_files = glob(data_dir + '/*')
for j in range(len(audio_files)):
    myaudio = AudioSegment.from_file("./sounds/{}".format(os.listdir('./sounds/')[j])) 
    chunk_length_ms = 10000
    chunks = make_chunks(myaudio, chunk_length_ms)

for i, chunk in enumerate(chunks):
    chunk_name = "{}{}.wav".format(os.listdir('./sounds/')[j], i)
    print("exporting", chunk_name)
    chunk.export(chunk_name, format="mp3")

exporting ducha.wav0.wav
exporting ducha.wav1.wav
exporting ducha.wav2.wav
exporting ducha.wav3.wav
exporting ducha.wav4.wav
exporting ducha.wav5.wav
exporting ducha.wav6.wav
exporting ducha.wav7.wav
exporting ducha.wav8.wav
exporting ducha.wav9.wav
exporting ducha.wav10.wav
exporting ducha.wav11.wav
exporting ducha.wav12.wav
exporting ducha.wav13.wav
exporting ducha.wav14.wav
exporting ducha.wav15.wav
exporting ducha.wav16.wav
exporting ducha.wav17.wav
exporting ducha.wav18.wav
exporting ducha.wav19.wav
exporting ducha.wav20.wav
exporting ducha.wav21.wav
exporting ducha.wav22.wav
exporting ducha.wav23.wav
exporting ducha.wav24.wav
exporting ducha.wav25.wav
exporting ducha.wav26.wav
exporting ducha.wav27.wav
exporting ducha.wav28.wav
exporting ducha.wav29.wav
exporting ducha.wav30.wav
exporting ducha.wav31.wav
exporting ducha.wav32.wav
exporting ducha.wav33.wav
exporting ducha.wav34.wav


In [22]:
def Pipeline():    
    data_dir = './sounds2'
    audio_files = glob(data_dir + '/*.wav')
    
    data = pd.DataFrame(columns = ['name','mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse'])
    
    lista = []
    for i in range(len(audio_files)):
        lista.append(os.listdir('./sounds2/')[i])
    data.name = lista
    
    lista2 = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        mfcc_aux = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        mfcc = np.ndarray.mean(mfcc_aux)
        lista2.append(mfcc)
    data.mfcc = lista2
    
    lista3 = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        scem = np.ndarray.mean(cent)
        lista3.append(scem)
    data.scem = lista3
    
    lista4 = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        S = np.abs(librosa.stft(y))
        contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
        scom = np.ndarray.mean(contrast)
        lista4.append(scom)
    data.scom = lista4
    
    lista5 = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        srom = np.ndarray.mean(rolloff)
        lista5.append(srom)
    data.srom = lista5
    
    lista6 = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        sbwm = np.ndarray.mean(spec_bw)
        lista6.append(sbwm)
    data.sbwm = lista6
    
    lista7 = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        hop_length = 512
        oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
        tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)

        ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
        ac_global = librosa.util.normalize(ac_global)

        tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr, hop_length=hop_length)[0]
        lista7.append(tempo)
    data.tempo = lista7
    
    lista8 = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        rmse_aux = librosa.feature.rms(y=y)
        rmse = np.ndarray.mean(rmse_aux)
        lista8.append(rmse)
    data.rmse = lista8
    
    momento = []

    for i in data['name']:
        if 'ducha' in i:
            momento.append('Ducha')
        elif 'cena' in i:
            momento.append('Cena')
        elif 'washing' in i:
            momento.append('Lavadora')
        elif 'vacuum' in i:
            momento.append('Aspiradora')
        elif 'shaver' in i:
            momento.append('Afeitadora')
        elif 'hairdryer' in i:
            momento.append('Secador_pelo')
        else:
            momento.append('Otro')

    data['momento_aux'] = momento
    #data['momento'] = 'Otro'
    data.drop(['name'], axis=1, inplace=True)
    
    return data

In [23]:
data = Pipeline()

In [18]:
data.shape

(35, 8)

In [19]:
data.dtypes

name      object
mfcc     float64
scem     float64
scom     float64
srom     float64
sbwm     float64
tempo    float64
rmse     float64
dtype: object

In [21]:
data.momento_aux.value_counts()

Ducha    35
Name: momento_aux, dtype: int64

In [42]:
data.to_csv('data_momento_aux_aux.csv', index=False)